In [91]:
import pandas as pd
import numpy as np
import csv
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_selection import SelectFpr
from sklearn.feature_selection import f_regression

## Data inlezen

In [9]:
data = pd.read_csv('recipes_v3.csv', sep=';')
len(data)

2023

In [10]:
 # Drop the duplicate recipes in the data
data.drop_duplicates()
len(data)

    # As you can see, the data has no duplicates

2023

## Data Pre-processen


In [11]:
    # Delete all rows with value NaN in columns
#data = data.fillna(0);
data = data.dropna()
len(data)

1473

In [12]:
rating_data = pd.DataFrame(data[['description', 'rating']])

In [13]:
X_train, X_test, Y_train, Y_test = train_test_split(rating_data['description'], rating_data['rating'], test_size=0.2, random_state=2432)

In [26]:
x_train_vect = CountVectorizer()
bow = x_train_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(bow)

X_test_counts = x_train_vect.transform(X_test)
x_test_tfidf = tfidf_transformer.transform(X_test_counts)

## F-test to prevent overfitting

In [123]:
uf = SelectFpr(f_regression)
fitted = uf.fit(x_train_tfidf, Y_train)
X_train_tfidf = fitted.transform(x_train_tfidf)
X_test_tfidf = fitted.transform(x_test_tfidf)

x_test_tfidf

<295x2629 sparse matrix of type '<class 'numpy.float64'>'
	with 5186 stored elements in Compressed Sparse Row format>

## Linear Regression

In [126]:

LR = LinearRegression()
LR.fit(X_train_tfidf, Y_train)
pred = LR.predict(X_test_tfidf)
LR.score(X_test_tfidf, Y_test)

-0.10532591626835086

## Making the labels binomial

In [135]:
y_train = list()
for i in Y_train: 
    if i <= 2.5:
        y_train.append(0)
    else:
        y_train.append(1)
        

 ## Multinomial Naive Bayes

In [141]:
Multi = MultinomialNB()
Multi.fit(X_train_tfidf, y_train)
pred = Multi.predict(X_test_tfidf)
accuracy = accuracy_score(pred, y_test)